In [1]:
import os
import time
os.path.getsize('predict-student-performance-from-game-play/train.csv')

4723977449

In [2]:
from dask import dataframe as dd
start = time.time()
dask_df = dd.read_csv('predict-student-performance-from-game-play/train.csv')
end = time.time()
print("Read csv with dask: ",(end-start),"sec")

Read csv with dask:  0.016386032104492188 sec


In [3]:
import pandas as pd
start = time.time()
df = pd.read_csv('predict-student-performance-from-game-play/train.csv')
end = time.time()
print("Read csv with pandas: ",(end-start),"sec")

Read csv with pandas:  81.71979117393494 sec


In [4]:
df = dd.read_csv('predict-student-performance-from-game-play/train.csv')

In [8]:
import modin.pandas as pd
import ray
ray.shutdown()
ray.init()
start = time.time()
df = pd.read_csv('predict-student-performance-from-game-play/train.csv')
end = time.time()
print("Read csv with modin and ray: ",(end-start),"sec")

2023-04-10 20:44:47,391	INFO worker.py:1553 -- Started a local Ray instance.


Read csv with modin and ray:  55.67519211769104 sec


In [10]:
len(df.index) # number of rows

26296946

In [11]:
len(df.columns) # number of columns

20

In [12]:
# remove special character
df.columns=df.columns.str.replace('[#,@,&]','')

In [13]:
#remove white space
df.columns = df.columns.str.replace(' ', '')

In [14]:
data=df.columns
data

Index(['session_id', 'index', 'elapsed_time', 'event_name', 'name', 'level',
       'page', 'room_coor_x', 'room_coor_y', 'screen_coor_x', 'screen_coor_y',
       'hover_duration', 'text', 'fqid', 'room_fqid', 'text_fqid',
       'fullscreen', 'hq', 'music', 'level_group'],
      dtype='object')

In [15]:
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re

In [16]:
%%writefile utility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re
def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.load(stream, Loader=yaml.Loader)
        except yaml.YAMLError as exc:
            logging.error(exc)

def col_header_val(df,table_config):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Writing utility.py


In [31]:
%%writefile store.yaml
file_type: csv
dataset_name: file
file_name: predict-student-performance-from-game-play/train
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - session_id
      - index
      - elapsed_time
      - event_name
      - name
      - level
      - page
      - room_coor_x
      - room_coor_y
      - screen_coor_x
      - screen_coor_y
      - hover_duration
      - text
      - fqid
      - room_fqid
      - text_fqid
      - fullscreen
      - hq
      - music

Overwriting store.yaml


In [32]:
# Reading config file
import utility as util
config_data = util.read_config_file('store.yaml')    

In [33]:
config_data

{'file_type': 'csv',
 'dataset_name': 'file',
 'file_name': 'predict-student-performance-from-game-play/train',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['session_id - index - elapsed_time - event_name - name - level - page - room_coor_x - room_coor_y - screen_coor_x - screen_coor_y - hover_duration - text - fqid - room_fqid - text_fqid - fullscreen - hq - music']}

In [34]:
from dask import dataframe as dd
df_sample = dd.read_csv('predict-student-performance-from-game-play/train.csv', delimiter=',')
df_sample.head()

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
0,20090312431273200,0,0,cutscene_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,0,0,1,0-4
1,20090312431273200,1,1323,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
2,20090312431273200,2,831,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
3,20090312431273200,3,1147,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,I gotta run to my meeting!,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
4,20090312431273200,4,1863,person_click,basic,0,NaN,-412.991405,-159.314686,381.0,494.0,NaN,"Can I come, Gramps?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4


In [35]:
#Reading the file using config file
file_type = config_data['file_type']
source_file = config_data['file_name'] + f'.{file_type}'

In [36]:
import pandas as pd
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df.head()

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
0,20090312431273200,0,0,cutscene_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,0,0,1,0-4
1,20090312431273200,1,1323,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
2,20090312431273200,2,831,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
3,20090312431273200,3,1147,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,I gotta run to my meeting!,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
4,20090312431273200,4,1863,person_click,basic,0,NaN,-412.991405,-159.314686,381.0,494.0,NaN,"Can I come, Gramps?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4


In [37]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['session_id', 'index', 'elapsed_time', 'event_name', 'name', 'level',
       'page', 'room_coor_x', 'room_coor_y', 'screen_coor_x', 'screen_coor_y',
       'hover_duration', 'text', 'fqid', 'room_fqid', 'text_fqid',
       'fullscreen', 'hq', 'music', 'level_group'],
      dtype='object')
columns of YAML are: ['session_id - index - elapsed_time - event_name - name - level - page - room_coor_x - room_coor_y - screen_coor_x - screen_coor_y - hover_duration - text - fqid - room_fqid - text_fqid - fullscreen - hq - music']


In [38]:
import datetime
import csv
import gzip

from dask import dataframe as dd
df = dd.read_csv('predict-student-performance-from-game-play/train.csv',delimiter=',')

# Write csv in gz format in pipe separated text file (|)
df.to_csv('train.csv.gz',
          sep='|',
          header=True,
          index=False,
          quoting=csv.QUOTE_ALL,
          compression='gzip',
          quotechar='"',
          doublequote=True,
          line_terminator='\n')

['c:\\Users\\zhanq\\Desktop\\NYU\\CS\\Machine Learning\\Project\\train.csv.gz\\00.part',
 'c:\\Users\\zhanq\\Desktop\\NYU\\CS\\Machine Learning\\Project\\train.csv.gz\\01.part',
 'c:\\Users\\zhanq\\Desktop\\NYU\\CS\\Machine Learning\\Project\\train.csv.gz\\02.part',
 'c:\\Users\\zhanq\\Desktop\\NYU\\CS\\Machine Learning\\Project\\train.csv.gz\\03.part',
 'c:\\Users\\zhanq\\Desktop\\NYU\\CS\\Machine Learning\\Project\\train.csv.gz\\04.part',
 'c:\\Users\\zhanq\\Desktop\\NYU\\CS\\Machine Learning\\Project\\train.csv.gz\\05.part',
 'c:\\Users\\zhanq\\Desktop\\NYU\\CS\\Machine Learning\\Project\\train.csv.gz\\06.part',
 'c:\\Users\\zhanq\\Desktop\\NYU\\CS\\Machine Learning\\Project\\train.csv.gz\\07.part',
 'c:\\Users\\zhanq\\Desktop\\NYU\\CS\\Machine Learning\\Project\\train.csv.gz\\08.part',
 'c:\\Users\\zhanq\\Desktop\\NYU\\CS\\Machine Learning\\Project\\train.csv.gz\\09.part',
 'c:\\Users\\zhanq\\Desktop\\NYU\\CS\\Machine Learning\\Project\\train.csv.gz\\10.part',
 'c:\\Users\\zhanq\\D

In [39]:
#number of files in gz format folder
import os
entries = os.listdir('train.csv.gz/')
for entry in entries:
    print(entry)

00.part
01.part
02.part
03.part
04.part
05.part
06.part
07.part
08.part
09.part
10.part
11.part
12.part
13.part
14.part
15.part
16.part
17.part
18.part
19.part
20.part
21.part
22.part
23.part
24.part
25.part
26.part
27.part
28.part
29.part
30.part
31.part
32.part
33.part
34.part
35.part
36.part
37.part
38.part
39.part
40.part
41.part
42.part
43.part
44.part
45.part
46.part
47.part
48.part
49.part
50.part
51.part
52.part
53.part
54.part
55.part
56.part
57.part
58.part
59.part
60.part
61.part
62.part
63.part
64.part
65.part
66.part
67.part
68.part
69.part
70.part
71.part
72.part


In [40]:
#size of the gz format folder
os.path.getsize('train.csv.gz')

32768